**CS 6307.501: Assignment 2**

**Submitted by: Anupa Desai and Lakshmipriya Narayanan**

Question 1

In [0]:
# Load the 'Sherlock Holmes' book from DBFS and take a look at it
book = sc.textFile("dbfs:/FileStore/sherlockholmes.txt")
book.take(20)

Out[8]: ['The Project Gutenberg eBook of The Adventures of Sherlock Holmes',
 '    ',
 'This ebook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this ebook or online',
 'at www.gutenberg.org. If you are not located in the United States,',
 'you will have to check the laws of the country where you are located',
 'before using this eBook.',
 '',
 'Title: The Adventures of Sherlock Holmes',
 '',
 'Author: Arthur Conan Doyle',
 '',
 'Release date: March 1, 1999 [eBook #1661]',
 '                Most recently updated: October 10, 2023',
 '',
 'Language: English',
 '',
 'Credits: an anonymous Project Gutenberg volunteer and Jose Menendez']

In [0]:
# Install the necessary nlp libraries and a pre-trained word vector used for named entities
%pip install spacy
!python -m spacy download en_core_web_md

Python interpreter will be restarted.
Python interpreter will be restarted.
     

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 33.5 MB 6.7 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-19e41578-ba6a-4259-a39d-8026a124bcc7/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [0]:
# Load spaCy library and the "en_core_web_md" model into nlp
import spacy
nlp_spacy = spacy.load("en_core_web_md")

# Function to extract named entities
def extract_named_entities(line_input):
    # Map to extract the text and remove whitespace
    # Filter the entities extracted by the map that belong to person, organizations, countries, cities, states, locations, etc.
    named_entities = map(lambda x: x.text.strip(), filter(lambda x: x.label_ in ("PERSON", "ORG", "GPE", "LOC", "NORP", "PRODUCT", "DATE", "MONEY", "FAC"), nlp_spacy(line_input).ents))

    return list(named_entities) # Return a list of entities


In [0]:
# Read the textfile into an RDD such that each line is a string input
lines_rdd = sc.textFile("dbfs:/FileStore/sherlockholmes.txt")

# Return an RDD where each element is a list of named entities from the line
entities_rdd = lines_rdd.map(extract_named_entities)

# Use flatMap to flatten the list of RDDs to individual RDDs
flat_entities_rdd = entities_rdd.flatMap(lambda x: x)

# Map each entity to a key value pair where key is the names entity and value is the number of occurences
kv_rdd = flat_entities_rdd.map(lambda entity: (entity, 1))

# Add up the occurences of each named entity and put it into the value
entity_counts_rdd = kv_rdd.reduceByKey(lambda x, y: x + y)

# Display the sorted RDDs in descending order
sorted_counts_rdd = entity_counts_rdd.sortBy(lambda x: -x[1])



In [0]:
# Display the named entities as pair RDDs (Key, Values)
sorted_counts_rdd.collect()

Out[3]: [('Holmes', 369),
 ('Watson', 81),
 ('Sherlock Holmes', 81),
 ('Lestrade', 37),
 ('Rucastle', 36),
 ('London', 36),
 ('Project Gutenberg', 36),
 ('McCarthy', 32),
 ('Baker Street', 25),
 ('I.', 24),
 ('Turner', 21),
 ('Arthur', 20),
 ('England', 20),
 ('Hunter', 19),
 ('Frank', 19),
 ('Toller', 15),
 ('Windibank', 15),
 ('Simon', 15),
 ('Mary', 14),
 ('German', 14),
 ('Roylott', 14),
 ('Neville St. Clair', 13),
 ('Hosmer Angel', 13),
 ('the day', 13),
 ('Wilson', 13),
 ('Bradstreet', 12),
 ('Merryweather', 12),
 ('Holder', 12),
 ('Irene Adler', 12),
 ('madam', 11),
 ('Horner', 11),
 ('American', 11),
 ('Hatherley', 11),
 ('Peterson', 11),
 ('Openshaw', 11),
 ('Lascar', 11),
 ('Alice', 11),
 ('Jones', 11),
 ('Lord St. Simon', 11),
 ('Foundation', 11),
 ('Briony Lodge', 10),
 ('the United States', 10),
 ('Ryder', 10),
 ('Project', 10),
 ('Horsham', 10),
 ('Moran', 10),
 ('St. Clair', 10),
 ('America', 10),
 ('Winchester', 10),
 ('yesterday', 9),
 ('Henry Baker', 9),
 ('Hosmer', 8

Question 2

In [0]:
# Install the nltk library
%pip install nltk


Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# Import the Natural Language ToolKit (nltk) library and the Regular Expression (re) library for string manipulation and matching. Import the 'stopwords' to make the remove stopwords function
import nltk
import re
from nltk.corpus import stopwords
from pyspark.context import SparkContext
import math
import functools
from collections import Counter

# Download the list of stopwords and the 'Punkt' tokenizer models to divide a text into a list of sentences or words.
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Out[1]: True

In [0]:
# Read the file into an RDD
plot_summaries = sc.textFile("dbfs:/FileStore/plot_summaries.txt")

# Split each line into 2 parts to separate the movie ID and teh corresponding plot based on the first tab
plot_summaries_rdd = plot_summaries.map(lambda line: line.split("\t", 1))

#  Filter the RDD to keep only lines with exactly two parts: (movieID, plot)
plot_summaries_rdd = plot_summaries_rdd.filter(lambda parts: len(parts) == 2)

# See the output 
plot_summaries_rdd.take(10)

Out[2]: [['23890098',
  "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all."],
 ['31186339',
  'The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively 

Removing stopwords using the nltk library

In [0]:
# Get English stopwords that we downloaded above. Use a set to make it faster to look up the words
stop_words = set(stopwords.words('english'))

# Define a function to remove stopwords and take a line as an input
def remove_stopwords(line):
    # Check if the line has exactly two parts (movieID and plot)
    if len(line) == 2:
        # If yes, split the plot into individial words, convert them to lowercase
        words = line[1].lower().split()
        
        # Now, remove the stopwords        
        filtered_words = list(filter(lambda word: word not in stop_words, words))
        
        # Reconstruct the plot without stopwords by joining into a single string
        filtered_summary = ' '.join(filtered_words)
        
        # Return the original movieID with the new plot
        return [line[0], filtered_summary]
    
    # If line does not have excatly 2 parts, then return the original line
    return line

# Apply the stopword removal function defined above to the RDD of plot summaries
plot_summaries_rdd_cleaned = plot_summaries_rdd.map(remove_stopwords)

# Display the first 10 cleaned RDD to see if function was applied
plot_summaries_rdd_cleaned.take(10)


Out[3]: [['23890098',
  'shlykov, hard-working taxi driver lyosha, saxophonist, develop bizarre love-hate relationship, despite prejudices, realize different all.'],
 ['31186339',
  'nation panem consists wealthy capitol twelve poorer districts. punishment past rebellion, district must provide boy girl ages 12 18 selected lottery annual hunger games. tributes must fight death arena; sole survivor rewarded fame wealth. first reaping, 12-year-old primrose everdeen chosen district 12. older sister katniss volunteers take place. peeta mellark, baker\'s son gave katniss bread starving, district 12 tribute. katniss peeta taken capitol, accompanied frequently drunk mentor, past victor haymitch abernathy. warns "career" tributes train intensively special academies almost always win. tv interview caesar flickerman, peeta unexpectedly reveals love katniss. outraged, believing ploy gain audience support, "sponsors" may provide in-games gifts food, medicine, tools. however, discovers peeta meant s

Calculating TF-IDF for one search term per line

In [0]:
# Read the search file for tf-idf (one search term per line) and the movie_metadata.tsv to connect the movieID with it's corresponding movie name
user_srch_terms_opl = sc.textFile("dbfs:/FileStore/search_terms_oneperline.txt")
movie_metadata = sc.textFile("dbfs:/FileStore/movie_metadata.tsv")

# Create a function to map the movie ID to movie name using the metadata file
def create_movie_name_map():    
    # Map movie metadata to (movie_id, movie_name) by splitting each line of the metadata file using 'tab'
    movie_names_rdd = movie_metadata.map(lambda line: line.split("\t"))
    
    # Since movie name is in the second column (index 1) and movie ID is in the first column (index 0) in the metadata file, we extract them and map them togther using the filter function and return the value.
    # Map each element of movie names RDD to a tuple (movie ID, movie name). If the line has fewer parts, then it maps to nothing and filter out no movies and finally pair RDD is (movie id, movie name)
    movie_name_map = movie_names_rdd.map(lambda parts: (parts[0], parts[2]) if len(parts) > 2 else (None, None)).filter(lambda x: x[0] is not None).collectAsMap()

    return movie_name_map

# Create movie name mapping for easier display where the dictionary that maps the movie ids to their corresponding names
MOVIE_NAMES = create_movie_name_map()

# Define a function to calculate the TF-IDF
def calc_tfidf(search_term):

    # Term Frequency (TF) Calculation takes the tuple(line) of documnet id and plot summary as the input
    def calc_tf(line):
        doc_id, summary = line

        # Convert the plot to lowercase and split into individual words
        words = summary.lower().split()
        
        # Count the total words and term occurrences of each search term in the search file for one term per line
        total_words = len(words)
        term_count = words.count(search_term.lower())
        
        # Compute term frequency (normalized) which is the ratio of occurence of a term to its total number of words
        tf = term_count / total_words if total_words > 0 else 0
        
        # Return the document id and its corresponding TF
        return (doc_id, tf)
    
    # Compute the term frequency for each document to help with IDF calculation below
    tf_rdd = plot_summaries_rdd_cleaned.map(calc_tf)
    
    # Inverse Document Frequency (IDF) Calculation
    def calc_idf(search_term):
        
        # Count the total number of documents
        total_docs = plot_summaries_rdd_cleaned.count()
        
        # Count the number of documents containing the serach term in the search file
        docs_with_term = plot_summaries_rdd_cleaned.filter(lambda line: search_term.lower() in line[1].lower()).count()
        
        # Compute IDF and avoid division by zero using the formula log (total docs/(1 + number of documents with the corresponding search term))
        idf = math.log(total_docs / (docs_with_term + 1))
        
        # Return the calculated IDF
        return idf
    
    # Compute the IDF for the corresponding search term
    idf = calc_idf(search_term)
    
    # TF-IDF Calculation with the tuple from above as the input
    def calc_tf_idf(tf_tuple):        
        doc_id, tf = tf_tuple
        # TF-IDF = TF * IDF
        return (doc_id, tf * idf)
    
    # Compute TF-IDF scores of each element in the RDD for TF
    tfidf_rdd = tf_rdd.map(calc_tf_idf)
    
    # Sort in descending order and get top 10 documents with their respective TF-IDF scores
    top_docs = tfidf_rdd.sortBy(lambda x: x[1], ascending=False).take(10)
    
    # Map the movie names with the scores above for display. Return 'NA' if the name of the movie is not found    
    results_combined = list(map(lambda x: (x[0], MOVIE_NAMES.get(x[0], "NA"), x[1]), top_docs))
        
    return results_combined

# Take a search term as user input and return the TF-IDF
def search_docs(term):    
    return calc_tfidf(term)

# Collect the search terms in the file and display to user
search_terms = user_srch_terms_opl.collect()
print("Search terms present in this file:", search_terms)

# User selects a genre and looks for this genre in the search file to calculate TF-IDF
user_term = "Romance"
top_docs = search_docs(user_term)

# Display results with movie name and TF-IDF score
print(f"\nTop 10 documents for genre '{user_term}':")
list(map(lambda doc: print(f"movie Name: {doc[1]}, TF-IDF: {doc[2]:.5f}"), top_docs))


Search terms present in this file: ['Funny ', 'Romance', 'thriller', 'Horror', 'Drama', 'Adventure', 'Classic', 'Musical', 'Mystery']

Top 10 documents for genre 'Romance':
movie Name: Office Lady Love Juice, TF-IDF: 0.36379
movie Name: That Dangerous Age, TF-IDF: 0.33072
movie Name: Dance Pretty Lady, TF-IDF: 0.30316
movie Name: Lovers and Liars, TF-IDF: 0.25985
movie Name: Moment by Moment, TF-IDF: 0.25985
movie Name: Raakh, TF-IDF: 0.24252
movie Name: Coming Back, TF-IDF: 0.24252
movie Name: Barefooted Youth, TF-IDF: 0.24252
movie Name: Yeh Dil Aap Ka Huwa, TF-IDF: 0.24252
movie Name: The Butler's in Love, TF-IDF: 0.21399
Out[4]: [None, None, None, None, None, None, None, None, None, None]

Calculating Cosine Similarity for a search query

In [0]:
# Read files - user query (with sentence per line for cosine similarity), 
# the original plot_summaries and the movies metadata that contains the movie names
# corresponding to the movieId
search_terms = sc.textFile("dbfs:/FileStore/search_terms.txt")
plot_summaries = sc.textFile("dbfs:/FileStore/plot_summaries.txt")

# define a function to remove the stopwords from the text
def remove_stopwords(text):
    # filter the words that are in the stop_words and convert the text to lower case and split 
    return ' '.join(list(filter(lambda word: word not in stop_words, text.lower().split())))

# define function to count the words using python function Counter
def calc_word_freq(words):
    return dict(Counter(words))

# define the function to calculate the cosine similarity using the arguments query (user defined) and document (plot_summaries)
def calc_cosine_similarity(query, document):
    # Compute the word count using the count function previously defined
    query_freq = calc_word_freq(query)
    doc_freq = calc_word_freq(document)
    
    # Get the set of unique words from both query and document
    all_words = set(query_freq.keys()).union(set(doc_freq.keys()))
    
    # functionally compute the dot product of the document frequency and query frequency 
    dot_prod = functools.reduce(lambda x, word: x + query_freq.get(word, 0) * doc_freq.get(word, 0),all_words,0)
    
    # functionally compute the magnitude of query frequency vector
    query_magnitude = functools.reduce(lambda x, count: x + count ** 2,query_freq.values(),0)
    
    # functionally compute the magnitude of document frequency
    doc_magnitude = functools.reduce(lambda x, count: x + count ** 2,doc_freq.values(),0)
    
    # Compute cosine similarity using the formule in lecture notes
    if query_magnitude == 0 or doc_magnitude == 0:
        return 0
    
    return dot_prod / (math.sqrt(query_magnitude) * math.sqrt(doc_magnitude))

# define a function to perform document search based on the user query
def search_documents(query):
    # remove the stop words from the user query
    cleaned_query = remove_stopwords(query)
    # for uniformity, convert all words to lower case and then split the lines into individual tokens
    query_words = cleaned_query.lower().split()
    
    # convert the plot_summaries into an rdd by splitting the original document on \t (separate the movie ID and the description)
    plot_rdd = plot_summaries.map(lambda line: line.split("\t", 1))
    # filter based on parts that have length 2
    plot_rdd = plot_rdd.filter(lambda parts: len(parts) == 2)
    
    # Remove all the stop words from the plot_rdd
    plot_rdd = plot_rdd.map(lambda line: (line[0], remove_stopwords(line[1])))
    
    # Compute cosine similarity for each document (each movie ID)
    def calc_similarity(line):
        # define the structure of the rdd: line contains the movie ID first and then the description/summary
        doc_id, summary = line
        # split the description to lower case for uniformity and then split it into individual words/tokens
        summary_words = summary.lower().split()
        
        # Compute cosine similarity between the query words and the summary words using the function we defined above
        similarity = calc_cosine_similarity(query_words, summary_words)
        
        return (doc_id, similarity)
    
    # Compute similarities as an rdd
    similarities_rdd = plot_rdd.map(calc_similarity)
    
    # Sort and get top 10 documents
    top_documents = similarities_rdd.sortBy(lambda x: x[1], ascending = False).take(10)
    
    # Combine results with movie names corresponding to the movie IDs
    return list(map(lambda x: (MOVIE_NAMES.get(x[0], "NA"), x[1]), top_documents))

# Predefined list of queries
queries = ["Funny movie with action scenes"]
search_queries = search_terms.collect()
print("Available Search Terms:", search_queries)

# Execute searches and display results
list(map(lambda query: (print(f"\nTop 10 movies for '{query}':"),list(map(lambda result: print(f"Movie Name: {result[0]}, Cosine Similarity: {result[1]:.5f}"), search_documents(query)))), queries))


Available Search Terms: ['Funny movie with action scenes', 'Romantic comedy', 'Sci-fi thriller', 'Horror movie with a twist', 'Animated family film', 'Drama based on true events', 'Adventure movie with a strong female lead', 'Classic black-and-white film', 'Musical with great dance numbers', 'Mystery with unexpected plot twists']

Top 10 movies for 'Funny movie with action scenes':
Movie Name: The Daredevil Men, Cosine Similarity: 0.33710
Movie Name: Chingari, Cosine Similarity: 0.29417
Movie Name: Murder at Midnight, Cosine Similarity: 0.28347
Movie Name: The Last Movie, Cosine Similarity: 0.27975
Movie Name: This Is Nollywood, Cosine Similarity: 0.27472
Movie Name: Action Man: Robot Atak, Cosine Similarity: 0.27280
Movie Name: Iruvattam Manavaatti, Cosine Similarity: 0.26941
Movie Name: Crayon Shin-chan: The Storm Called The Jungle, Cosine Similarity: 0.24845
Movie Name: Odessa in Flames, Cosine Similarity: 0.24749
Movie Name: Josettante Hero, Cosine Similarity: 0.24660
Out[5]: [(Non